# Aurite API Agent: Functional Testing

This notebook is for functionally testing the `Aurite API Agent`. This agent uses an MCP server that wraps the framework's own API, allowing it to create and execute other components.

## 1. Setup
### Install the `aurite` Python package
First, ensure you have the `aurite-agents` package installed. If you are running this from the source repository, you can install it in editable mode with `pip install -e .` from the root directory.

In [ ]:
%pip install -q -U "aurite-agents[default]"

### Configure Your API Key
The `Aurite API Agent` needs an API key to communicate with the framework's API. For security, we'll load this from an environment variable.

In [ ]:
import os
from getpass import getpass

try:
    from dotenv import load_dotenv
    load_dotenv()  # Load environment variables from a .env file if it exists
except ImportError:
    print("dotenv not installed, skipping. Please ensure API_KEY is set manually.")

if "API_KEY" not in os.environ:
    os.environ["API_KEY"] = getpass("Enter your Aurite API key: ")

## 2. Initialize Aurite & Define Components

Next, we'll initialize the `Aurite` framework. Instead of loading from a file, we will define our `ClientConfig` and `AgentConfig` directly in Python.

In [ ]:
from aurite import Aurite, AgentConfig, ClientConfig

aurite = Aurite()

# This command starts up the necessary background services.
await aurite.initialize()

### Define the API MCP Server

Here, we define the configuration for our API-wrapping MCP server. We use an absolute path to the server script.

In [ ]:
api_server_config = ClientConfig(
    name="api_mcp_server",
    server_path="/home/wilcoxr/workspace/aurite/aurite-agents/src/aurite/components/servers/orchestration/api_mcp_server.py",
    capabilities=["tools"],
    timeout=30.0
)

await aurite.register_client(api_server_config)
print("✅ API MCP Server registered successfully!")

### Define the Aurite API Agent

Now, we create the agent that will use our new server. Its system prompt instructs it to use the tools provided.

In [ ]:
api_agent_config = AgentConfig(
    name="Aurite API Agent",
    llm_config_id="anthropic_claude_3_haiku", # Assumes a default LLM is available
    mcp_servers=["api_mcp_server"],
    system_prompt="You are an agent that can interact with the Aurite framework's API. Use the available tools to manage components. CRITICAL: You must use the exact tool names provided."
)

await aurite.register_agent(api_agent_config)
print("✅ Aurite API Agent registered successfully!")

### Helper for Displaying Output

In [ ]:
from IPython.display import display, Markdown
import json

def display_agent_response(agent_name: str, query: str, response: str):
    """Formats and displays the agent's response in a structured Markdown block."""
    
    # Try to pretty-print if the response is a JSON string
    try:
        parsed_json = json.loads(response)
        formatted_response = f"```json\n{json.dumps(parsed_json, indent=2)}\n```"
    except (json.JSONDecodeError, TypeError):
        formatted_response = response

    output = f"""
    <div style=\"border: 1px solid #D1D5DB; border-radius: 8px; margin-top: 20px; font-family: sans-serif; box-shadow: 0 4px 6px rgba(0,0,0,0.05);\">
      <div style=\"background-color: #F3F4F6; padding: 10px 15px; border-bottom: 1px solid #D1D5DB; border-radius: 8px 8px 0 0;\">
        <h3 style=\"margin: 0; font-size: 16px; color: #1F2937; display: flex; align-items: center;\">
          <span style=\"margin-right: 8px;\">🤖</span>
          Agent Response: <code style=\"background-color: #E5E7EB; color: #374151; padding: 2px 6px; border-radius: 4px; margin-left: 8px;\">{agent_name}</code>
        </h3>
      </div>
      <div style=\"padding: 15px;\">
        <p style=\"margin: 0 0 10px 0; color: #6B7280; font-size: 14px;\">
          <strong>Your Query:</strong>
        </p>
        <p style=\"background-color: #F9FAFB; margin: 0 0 15px 0; color: #1F2937; border: 1px solid #E5E7EB; border-left: 3px solid #9CA3AF; padding: 10px 12px; border-radius: 4px;\">
          <em>\"{query}\"</em>
        </p>
        <hr style=\"border: none; border-top: 1px dashed #D1D5DB; margin-bottom: 15px;\">
        <p style=\"margin: 0 0 10px 0; color: #6B7280; font-size: 14px;\">
          <strong>Result:</strong>
        </p>
        <div style=\"background-color: #FFFFFF; padding: 15px; border-radius: 5px; border: 1px solid #E5E7EB; color: #1F2937; font-size: 15px; line-height: 1.6;\">
          {formatted_response}
        </div>
      </div>
    </div>
    """
    display(Markdown(output))

## 3. Run the Test Case

Now, we'll run the agent with a two-step command: first create a new agent, then execute it.

In [ ]:
user_message = "First, create an agent named 'notebook-test-agent' with the system prompt 'I am a test agent created from a notebook.' Then, execute that agent with the input 'Say hello.'"

# Run the agent with our query
agent_result = await aurite.run_agent(
    agent_name="Aurite API Agent",
    user_message=user_message
)

# Display the final response
display_agent_response(
    agent_name="Aurite API Agent",
    query=user_message,
    response=agent_result.primary_text
)